In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Notebooks/Github/twitter-sentiment-analysis/

/content/drive/MyDrive/Notebooks/Github/twitter-sentiment-analysis


In [4]:
import numpy as np
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm
tqdm.pandas()


In [5]:
def prepare_tweets_df(df, stock_sym):
    tweets_dict = {
        'stock': [],
        'date': [],
        'tweets': []
    }
    for _, row in df.iterrows():
        date = row['date']
        tweets = row.iloc[2:61].values
        # print(len(tweets))
        tweets_dict['tweets'].extend(tweets)
        tweets_dict['stock'].extend([stock_sym for i in range(0, len(tweets))])
        tweets_dict['date'].extend([date for i in range(0, len(tweets))])
    return pd.DataFrame(tweets_dict)

In [6]:
tweet_files_path = os.getcwd() + '/dataset/Tweets/'
stocks_files_path = os.getcwd() + '/dataset/Stocks/'

stock_names = ['AAPL', 'AMT', 'AMZN', 'APRN', 'C', 'DIS', 'FB', 'GE', 'GOOGL', 'HTZ', 'LUV', 'MSFT', 'NFLX', 'SBUX', 'TSLA']

tweets_df_list = []
for stock_name in stock_names:
    tweets_df_list.append(prepare_tweets_df(pd.read_csv(tweet_files_path + stock_name + '.csv'), stock_name))

tweets_df = pd.concat(tweets_df_list, axis = 0)

# tweets_df = pd.concat([prepare_tweets_df(pd.read_csv(tweet_files_path + stock_name + '.csv'), stock_name) for stock_name in stock_names], axis = 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
tweets_df.sample(20)
print(tweets_df.shape)
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
# print(tweets_df.duplicated().sum())
print(len(tweets_df)-len(tweets_df.drop_duplicates()))
tweets_df = tweets_df.drop_duplicates()
print(tweets_df.isna().sum())
print(tweets_df.shape)

(1269444, 3)
stock           0
date            0
tweets    1002313
dtype: int64
17936
stock     0
date      0
tweets    0
dtype: int64
(249195, 3)


In [8]:
# Due to incorrect formatting of the source csv files, some strings are available in date column. 
# So we are forcing those values to be NaT
# Also dropping those rows
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
print(tweets_df.shape)
tweets_df = tweets_df.reset_index(drop = True)

stock       0
date      170
tweets      0
dtype: int64
(249025, 3)


In [9]:
tweets_df.to_csv('./dataset/Tweets/tweets-roberta.csv', index = False)

In [10]:
# !pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.8 MB/s 
     |████████████████████████████████| 95 kB 5.7 MB/s 
     |████████████████████████████████| 115 kB 77.6 MB/s 
     |████████████████████████████████| 212 kB 85.6 MB/s 
     |████████████████████████████████| 163 kB 80.9 MB/s 
     |████████████████████████████████| 127 kB 70.9 MB/s 
     |████████████████████████████████| 115 kB 72.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [11]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='./dataset/Tweets/tweets-roberta.csv', split='train')
dataset[100]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e980d31f4468c654/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


{'stock': 'AAPL',
 'date': '2019-07-16',
 'tweets': "Maps: What's New in iOS 13  http://dlvr.it/R8XYGN\xa0 $AAPL  pic.twitter.com/t2NPYX6kkE"}

In [12]:
# !pip install -q -U transformers
# !pip install -U torch torchvision torchaudio
# !pip install -U emoji

     |████████████████████████████████| 5.3 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 75.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████████████████████████▏  | 812.9 MB 112.7 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 890.2 MB 6.3 kB/s 
     |████████████████████████████████| 24.3 MB 74.4 MB/s 
     |████████████████████████████████| 4.2 MB 66.2 MB/s 
     |████████████████████████████████| 557.1 MB 11 kB/s 
     |████████████████████████████████| 21.0 MB 53.3 MB/s 
     |████████████████████████████████| 849 kB 74.4 MB/s 
     |████████████████████████████████| 317.1 MB 30 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does no

In [13]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
# https://huggingface.co/zhayunduo/roberta-base-stocktwits-finetuned?text=I+like+you.+I+love+you
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import emoji
import numpy as np
from scipy.special import softmax
import re

MODEL = f"zhayunduo/roberta-base-stocktwits-finetuned"
# config = AutoConfig.from_pretrained(MODEL)
tokenizer = RobertaTokenizer.from_pretrained(MODEL)
model = RobertaForSequenceClassification.from_pretrained(MODEL)
model.to(device)
device

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/782 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

device(type='cuda')

In [15]:
def preprocess_dataset(row):
  texts = row['tweets']
  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))

  row['tweets'] = texts.strip()
  return row

In [16]:
preprocessed_dataset =  dataset.map(preprocess_dataset)

  0%|          | 0/249025 [00:00<?, ?ex/s]

In [17]:
def tokenize_function(row):
  return tokenizer(row['tweets'], return_tensors='pt', truncation=True, padding = 'max_length', max_length = 512).to(device)

In [18]:
tokenized_datasets = preprocessed_dataset.map(tokenize_function, batched=True)

  0%|          | 0/250 [00:00<?, ?ba/s]

In [19]:
len(tokenized_datasets)

249025

In [20]:
# I am changing the type of tensor columns (input_ids, attention_mask) to torch type
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')


In [21]:
dataloader = torch.utils.data.DataLoader(tokenized_datasets, batch_size=32)

In [22]:
scores_dict = {
    'bearish': [],
    'bullish': []
}

for batch in tqdm(dataloader):
  # with torch.no_grad():        
  outputs = model(**batch)
  logits = outputs.logits
  logits = logits.detach().cpu().numpy()
  for logit in logits:
    scores = softmax(logit)
    scores_dict['bearish'].append(scores[0])
    scores_dict['bullish'].append(scores[1])



  0%|          | 0/7783 [00:00<?, ?it/s]

In [23]:
scores_df = pd.DataFrame(scores_dict)
print(scores_df.shape)
display(scores_df.head())

(249025, 2)


,bearish,bullish
0,0.001388,0.998612
1,0.001409,0.998591
2,0.001429,0.998571
3,0.001382,0.998618
4,0.001471,0.998529


In [24]:
len(tweets_df)

249025

In [25]:
final_df = pd.concat([tweets_df, scores_df], axis = 1)
final_df.to_csv('./dataset/Tweets/tweets-sentiment-roberta.csv', index = False)